In [15]:
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf
#Import all the dependencies
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
# import the metrics class
from sklearn import metrics
import matplotlib.pyplot as plt
import warnings
from nltk.tokenize import word_tokenize

In [2]:
warnings.filterwarnings('ignore')
#warnings.filterwarnings(action='once')

In [3]:
#read in the preprocessed data as csv using pandas
df = pd.read_csv("preprocessed_data.csv")
df.head() #take a look

,filepath,age,gender,mmse,pause1,pause2,pause3,count_unintelligible,count_trailing,count_repetitions,category,data
0,138-1.cha,68.0,female,28.0,0,0,1,0,1,3,Control,there's a cookie jar on the shelf . and the ...
1,631-0.cha,74.0,male,29.0,0,0,0,0,0,0,Control,the kids are in the cookies . the stool is fa...
2,182-3.cha,NaN,male,NaN,0,0,0,0,1,0,Control,well Johnny's falling off the stool . the boy...
3,121-0.cha,70.0,female,30.0,0,1,0,0,0,0,Control,the boy is taking a cookie outof the cookie ...
4,142-3.cha,61.0,female,30.0,0,0,0,0,0,0,Control,well the water's running over on the floor ....


In [4]:
df_sub = df.loc[df.category.isin([ "Control", "ProbableAD"])] #read the important identity
df_sub

,filepath,age,gender,mmse,pause1,pause2,pause3,count_unintelligible,count_trailing,count_repetitions,category,data
0,138-1.cha,68.0,female,28.0,0,0,1,0,1,3,Control,there's a cookie jar on the shelf . and the ...
1,631-0.cha,74.0,male,29.0,0,0,0,0,0,0,Control,the kids are in the cookies . the stool is fa...
2,182-3.cha,NaN,male,NaN,0,0,0,0,1,0,Control,well Johnny's falling off the stool . the boy...
3,121-0.cha,70.0,female,30.0,0,1,0,0,0,0,Control,the boy is taking a cookie outof the cookie ...
4,142-3.cha,61.0,female,30.0,0,0,0,0,0,0,Control,well the water's running over on the floor ....
5,086-2.cha,NaN,female,NaN,1,1,0,0,0,1,Control,well the boy has climbed up on the stool to g...
6,267-2.cha,54.0,female,30.0,2,0,0,0,0,1,Control,mother is drying the dishes and looking out t...
7,118-4.cha,NaN,female,NaN,1,0,0,0,0,0,Control,well the water's running over outta out the s...
8,166-2.cha,NaN,male,NaN,1,0,0,0,0,0,Control,all of the things that I see going on ? well ...
9,105-1.cha,57.0,female,27.0,0,0,0,0,0,0,Control,well the mother is drying the dishes . the s...


In [5]:
df_sub["data"].tolist() #collect all interview transcripts into a list

["  there's a cookie jar on the shelf . and the little boy's up on a stool . and the stool is about to fall he's about to fall . the stool's about to turn over . little girl in the picture's reaching for the cookie . mother is washing dishes . the water's running and running over onto the floor . she she has a plate in her hand and a towel . she's drying a plate . two cups are sitting on a counter and also a plate . you can see out through the kitchen window to another part of the house and a a walkway away from the house . you can see a window across in the other part of the house, the roof with a tree, that there are there are curtains on the windows .  I think that's did I cover it all ? yeah . ",
 " the kids are in the cookies . the stool is falling over . the mother's spilling the water and also drying the dishes . and the wind might be blowing the curtains . and the water's running . I can't tell is anything going on outside or not . I guess that's all I see . that's not very man

In [6]:
#data cleaning
REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]") #delete all puntuations
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)") #put space back in
NO_SPACE = ""
SPACE = " "

#data cleaning function
def preprocess_interviews(interviews):
    interviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in interviews]
    interviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in interviews]
    
    return interviews #return processed data

interviews_train_clean = preprocess_interviews(df_sub["data"].tolist()) #put all cleaned data in a list

In [7]:
interviews_train_clean

['  theres a cookie jar on the shelf  and the little boys up on a stool  and the stool is about to fall hes about to fall  the stools about to turn over  little girl in the pictures reaching for the cookie  mother is washing dishes  the waters running and running over onto the floor  she she has a plate in her hand and a towel  shes drying a plate  two cups are sitting on a counter and also a plate  you can see out through the kitchen window to another part of the house and a a walkway away from the house  you can see a window across in the other part of the house the roof with a tree that there are there are curtains on the windows   i think thats did i cover it all  yeah  ',
 ' the kids are in the cookies  the stool is falling over  the mothers spilling the water and also drying the dishes  and the wind might be blowing the curtains  and the waters running  i cant tell is anything going on outside or not  i guess thats all i see  thats not very many  ',
 ' well johnnys falling off th

In [8]:
elmo_vectors = []

In [9]:
train_text = [[x] for x in interviews_train_clean]

In [10]:
train_text

[['  theres a cookie jar on the shelf  and the little boys up on a stool  and the stool is about to fall hes about to fall  the stools about to turn over  little girl in the pictures reaching for the cookie  mother is washing dishes  the waters running and running over onto the floor  she she has a plate in her hand and a towel  shes drying a plate  two cups are sitting on a counter and also a plate  you can see out through the kitchen window to another part of the house and a a walkway away from the house  you can see a window across in the other part of the house the roof with a tree that there are there are curtains on the windows   i think thats did i cover it all  yeah  '],
 [' the kids are in the cookies  the stool is falling over  the mothers spilling the water and also drying the dishes  and the wind might be blowing the curtains  and the waters running  i cant tell is anything going on outside or not  i guess thats all i see  thats not very many  '],
 [' well johnnys falling o

In [82]:
train_text_part = []
for i in range(400,477): #100 - 199 #左闭右开 #use 477 to include all 476 texts
    train_text_part.append(train_text[i])

In [83]:
train_text_part[76]

['  i see the little boy stealing cookies from the cookie jar  and the little girls he gave some to the little girl and shes eating some of the cookies  and i guess this is mama and shes washing the dishes  and she dropped a dish  no no she didnt drop a dish  the the water that shes washing the dishes with she let run  and its overflown overflown overflown  that doesnt that doesnt sound right  did it we forgot to turn off the spigot  and so the water is running off into the onto the floor here  and mom apparently is washing the dishes  and heres this little boy stealing the cookie cookies  hes gonna gonna fall because his the the cookies jar or the bench s the four legged stool whatever it is is is gonna fall over with him and the cookie jar  and mamas drying the dishes as usual for mamas if they dont have a husband that dries them or washes them or whatever  lets see now  i guess theres more things im sposta see  lets see here now  oh and the the water is flowing out of the of the sin

In [84]:
len(train_text_part)

77

In [85]:
#To make tf 2.0 compatible with tf1.0 code, we disable the tf2.0 functionalities
tf.disable_eager_execution()

In [86]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [87]:
# just a random sentence

sentences = train_text_part



In [88]:
sentences

[[' oh my  you mean how much or just the ones   theres the sink is overflowing while shes wiping a plate and not looking  the little boy is hungry  he wants to get some cookies  and hes standing on a stool  and its tipping over  and i dont know whether hes gonna bring the cookie jar with him or what  and the little girl is holding up her hand wanting some  and hes gonna give to him  but i dont know if she has a finger to her mouth but and she knows that hes gonna be tipping over it looks like to me  or maybe shes just not noticing it and making her mouth get ready for this good cookie  i dont know which way it is for the little girl  because shes smiling  i think if she saw him falling over shed do something to to help him so he wouldnt get hurt  imean shed grab it or something  but shes just looking up at him youknow looking up at those good cookies and waiting for them to come to her hand  but this little finger in her mouth is im not quite sure what it means  i think it means that i

In [89]:
def get_elmo_vectors(x):
  embeddings = elmo(x, signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [90]:
# Extract ELMo embeddings
elmo_vectors_part = [get_elmo_vectors(x) for x in sentences]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [91]:
len(elmo_vectors_part)

77

In [92]:
elmo_vectors_part

[array([[-0.10673783, -0.14773004, -0.02326553, ...,  0.00377272,
          0.66122603,  0.06845764]], dtype=float32),
 array([[ 0.08682484, -0.03644485, -0.01204507, ...,  0.03307454,
          0.46138728,  0.04769931]], dtype=float32),
 array([[ 0.2013614 , -0.19664705,  0.00282143, ...,  0.07228779,
          0.38507906,  0.15444453]], dtype=float32),
 array([[-0.03976833, -0.08882086,  0.02817707, ..., -0.07872383,
          0.5610051 , -0.00468263]], dtype=float32),
 array([[-0.01358772, -0.11245718,  0.04418973, ..., -0.07018895,
          0.54916835,  0.04642474]], dtype=float32),
 array([[ 0.15392321, -0.17527819,  0.11936738, ...,  0.00409883,
          0.49386704, -0.03748539]], dtype=float32),
 array([[-0.11500037, -0.1626498 , -0.09898206, ...,  0.15219659,
          0.5476619 , -0.01862704]], dtype=float32),
 array([[-0.10729565,  0.06626553, -0.0543077 , ...,  0.07841564,
          0.4479375 ,  0.0481075 ]], dtype=float32),
 array([[-0.09508482, -0.0584964 , -0.01779963, 

In [93]:
elmo_vectors = elmo_vectors+elmo_vectors_part

In [94]:
len(elmo_vectors)

476

In [95]:
print("Shape of each sentence embedding:")
for x in elmo_vectors:
    print(x.shape)

Shape of each sentence embedding:
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)
(1, 10

In [110]:
len(elmo_vectors)

476

In [111]:
elmo_vecs = []
for sublist in elmo_vectors:
    for item in sublist:
        elmo_vecs.append(item)

In [112]:
elmo_vecs

[array([ 0.25159258,  0.02076257, -0.00920534, ...,  0.07811424,
         0.5708164 ,  0.00484926], dtype=float32),
 array([ 0.2540095 , -0.14774652,  0.04500685, ...,  0.08484393,
         0.52722937, -0.00663076], dtype=float32),
 array([-0.0998107 , -0.03377883, -0.0558092 , ..., -0.004955  ,
         0.44657955,  0.16277756], dtype=float32),
 array([0.1579949 , 0.06644518, 0.0584579 , ..., 0.15729694, 0.5598118 ,
        0.03759524], dtype=float32),
 array([-0.08112554, -0.07984114, -0.0194974 , ..., -0.0285372 ,
         0.5952    ,  0.00599547], dtype=float32),
 array([ 0.00062499,  0.05985605, -0.07640619, ..., -0.00217756,
         0.5873181 , -0.11979379], dtype=float32),
 array([ 0.00806571, -0.13907361, -0.1223456 , ...,  0.23042507,
         0.49992394,  0.23734991], dtype=float32),
 array([-0.01413198,  0.07396255, -0.11735865, ..., -0.04683429,
         0.53032076,  0.06486255], dtype=float32),
 array([ 0.02811883,  0.03255862, -0.02039186, ...,  0.08681343,
         0.45

In [113]:
column_names = ["elmov"+str(i) for i in range(1024)]
column_names

['elmov0',
 'elmov1',
 'elmov2',
 'elmov3',
 'elmov4',
 'elmov5',
 'elmov6',
 'elmov7',
 'elmov8',
 'elmov9',
 'elmov10',
 'elmov11',
 'elmov12',
 'elmov13',
 'elmov14',
 'elmov15',
 'elmov16',
 'elmov17',
 'elmov18',
 'elmov19',
 'elmov20',
 'elmov21',
 'elmov22',
 'elmov23',
 'elmov24',
 'elmov25',
 'elmov26',
 'elmov27',
 'elmov28',
 'elmov29',
 'elmov30',
 'elmov31',
 'elmov32',
 'elmov33',
 'elmov34',
 'elmov35',
 'elmov36',
 'elmov37',
 'elmov38',
 'elmov39',
 'elmov40',
 'elmov41',
 'elmov42',
 'elmov43',
 'elmov44',
 'elmov45',
 'elmov46',
 'elmov47',
 'elmov48',
 'elmov49',
 'elmov50',
 'elmov51',
 'elmov52',
 'elmov53',
 'elmov54',
 'elmov55',
 'elmov56',
 'elmov57',
 'elmov58',
 'elmov59',
 'elmov60',
 'elmov61',
 'elmov62',
 'elmov63',
 'elmov64',
 'elmov65',
 'elmov66',
 'elmov67',
 'elmov68',
 'elmov69',
 'elmov70',
 'elmov71',
 'elmov72',
 'elmov73',
 'elmov74',
 'elmov75',
 'elmov76',
 'elmov77',
 'elmov78',
 'elmov79',
 'elmov80',
 'elmov81',
 'elmov82',
 'elmov83',
 '

In [114]:
len(column_names)

1024

In [115]:
elmovecs_df = pd.DataFrame(elmo_vecs)

In [ ]:
elmovecs_df.columns = column_names

In [118]:
elmovecs_df

,elmov0,elmov1,elmov2,elmov3,elmov4,elmov5,elmov6,elmov7,elmov8,elmov9,...,elmov1014,elmov1015,elmov1016,elmov1017,elmov1018,elmov1019,elmov1020,elmov1021,elmov1022,elmov1023
0,0.251593,0.020763,-0.009205,0.002106,0.136264,0.287263,-0.055806,-0.051846,0.068617,-0.123318,...,-0.019371,0.225851,0.057623,0.005713,0.160238,0.334751,0.126103,0.078114,0.570816,0.004849
1,0.254009,-0.147747,0.045007,0.089490,0.177276,0.167827,-0.026437,-0.158093,0.053951,-0.100942,...,0.089039,0.227415,0.031419,0.032353,-0.008896,0.335890,-0.050735,0.084844,0.527229,-0.006631
2,-0.099811,-0.033779,-0.055809,-0.012933,0.091413,0.082503,-0.057615,0.066562,-0.346306,-0.075564,...,-0.036673,0.320153,-0.098816,0.176296,0.183647,0.204000,0.239201,-0.004955,0.446580,0.162778
3,0.157995,0.066445,0.058458,0.194358,0.020746,0.091865,0.079901,0.241131,0.054250,-0.218863,...,0.005955,0.334145,0.105894,0.195061,0.057635,0.185550,0.092908,0.157297,0.559812,0.037595
4,-0.081126,-0.079841,-0.019497,0.037332,0.256142,0.153642,-0.001719,-0.004721,-0.217459,-0.135696,...,-0.034988,0.212992,-0.112018,0.076137,0.045127,0.190258,0.003921,-0.028537,0.595200,0.005995
5,0.000625,0.059856,-0.076406,-0.010028,0.146430,0.017271,0.051622,-0.065525,-0.153476,-0.111023,...,-0.112861,0.140078,0.110475,0.282923,0.071078,0.302004,-0.006380,-0.002178,0.587318,-0.119794
6,0.008066,-0.139074,-0.122346,0.094338,0.005450,0.139544,-0.101851,-0.150143,0.014084,-0.093273,...,0.143058,0.242639,-0.264474,0.079842,0.055345,0.165843,0.178632,0.230425,0.499924,0.237350
7,-0.014132,0.073963,-0.117359,0.017517,0.144658,0.095344,-0.102162,0.015073,-0.245441,-0.001322,...,0.061464,0.246043,-0.043044,0.108412,0.163218,0.211117,0.056776,-0.046834,0.530321,0.064863
8,0.028119,0.032559,-0.020392,0.023757,0.114377,0.037145,0.035356,0.123515,-0.014918,-0.130653,...,0.027903,0.311943,0.029258,-0.015005,0.104177,0.253527,0.103794,0.086813,0.454941,0.118941
9,-0.037541,-0.032611,-0.024463,-0.087944,0.159127,0.005041,0.071873,0.074638,-0.061838,-0.100274,...,-0.032514,0.184322,0.100605,0.053639,0.073916,0.276412,0.055412,0.027214,0.450903,0.109371


In [119]:
elmovecs_df.to_csv('elmo_vectors.csv')

In [120]:
elmovecs_df = pd.read_csv("elmo_vectors.csv")
elmovecs_df

,elmov0,elmov1,elmov2,elmov3,elmov4,elmov5,elmov6,elmov7,elmov8,elmov9,...,elmov1014,elmov1015,elmov1016,elmov1017,elmov1018,elmov1019,elmov1020,elmov1021,elmov1022,elmov1023
0,0.251593,0.020763,-0.009205,0.002106,0.136264,0.287263,-0.055806,-0.051846,0.068617,-0.123318,...,-0.019371,0.225851,0.057623,0.005713,0.160238,0.334751,0.126103,0.078114,0.570816,0.004849
1,0.254009,-0.147747,0.045007,0.089490,0.177276,0.167827,-0.026437,-0.158093,0.053951,-0.100942,...,0.089039,0.227415,0.031419,0.032353,-0.008896,0.335890,-0.050735,0.084844,0.527229,-0.006631
2,-0.099811,-0.033779,-0.055809,-0.012933,0.091413,0.082503,-0.057615,0.066562,-0.346306,-0.075564,...,-0.036673,0.320153,-0.098816,0.176296,0.183647,0.204000,0.239201,-0.004955,0.446580,0.162778
3,0.157995,0.066445,0.058458,0.194358,0.020746,0.091865,0.079901,0.241131,0.054250,-0.218863,...,0.005955,0.334145,0.105894,0.195061,0.057635,0.185550,0.092908,0.157297,0.559812,0.037595
4,-0.081126,-0.079841,-0.019497,0.037332,0.256142,0.153642,-0.001719,-0.004721,-0.217459,-0.135696,...,-0.034988,0.212992,-0.112018,0.076137,0.045127,0.190258,0.003921,-0.028537,0.595200,0.005995
5,0.000625,0.059856,-0.076406,-0.010028,0.146430,0.017271,0.051622,-0.065525,-0.153476,-0.111023,...,-0.112861,0.140078,0.110475,0.282923,0.071078,0.302004,-0.006380,-0.002178,0.587318,-0.119794
6,0.008066,-0.139074,-0.122346,0.094338,0.005450,0.139544,-0.101851,-0.150143,0.014084,-0.093273,...,0.143058,0.242639,-0.264474,0.079842,0.055345,0.165843,0.178632,0.230425,0.499924,0.237350
7,-0.014132,0.073963,-0.117359,0.017517,0.144658,0.095344,-0.102162,0.015073,-0.245441,-0.001322,...,0.061464,0.246043,-0.043044,0.108412,0.163218,0.211117,0.056776,-0.046834,0.530321,0.064863
8,0.028119,0.032559,-0.020392,0.023757,0.114377,0.037145,0.035356,0.123515,-0.014918,-0.130653,...,0.027903,0.311943,0.029258,-0.015005,0.104177,0.253527,0.103794,0.086813,0.454941,0.118941
9,-0.037541,-0.032611,-0.024463,-0.087944,0.159127,0.005041,0.071873,0.074638,-0.061838,-0.100274,...,-0.032514,0.184322,0.100605,0.053639,0.073916,0.276412,0.055412,0.027214,0.450903,0.109371


In [121]:
docvecs_df = pd.read_csv("doc2vec.csv")
docvecs_df

,docvecs0,docvecs1,docvecs2,docvecs3,docvecs4,docvecs5,docvecs6,docvecs7,docvecs8,docvecs9,...,docvecs990,docvecs991,docvecs992,docvecs993,docvecs994,docvecs995,docvecs996,docvecs997,docvecs998,docvecs999
0,-0.151947,-0.371280,-0.316788,0.218896,0.075059,0.085084,-0.014939,-0.332905,0.460935,-0.375095,...,0.003639,0.151513,-0.213763,0.260721,0.352837,0.141391,0.067808,0.140375,0.145603,-0.112032
1,0.027595,-0.244941,-0.015700,0.170830,0.000994,0.092973,-0.060648,-0.317840,-0.181011,0.074971,...,0.105571,0.137212,-0.395745,0.108204,-0.104591,0.033388,0.379854,-0.120298,0.210647,0.006663
2,-0.366985,0.126893,-0.169838,0.375112,-0.321718,0.236652,0.260350,-0.123940,0.463862,-0.413141,...,0.007199,-0.388843,-0.780460,-0.251663,0.107677,0.211592,0.313357,0.183024,0.260902,0.143484
3,0.254116,0.496882,-0.460124,0.546128,-0.271839,0.488808,0.580052,-0.058533,0.128703,0.862113,...,-0.167442,-0.320421,-0.371582,-0.371044,-0.115144,-0.122459,0.329042,-0.190539,-0.113650,0.227231
4,-0.267923,0.645562,0.122693,-0.173702,0.362269,0.026052,0.366241,-0.383295,-0.076927,-0.022577,...,-0.116609,-0.449935,-0.286522,0.017240,-0.300730,-0.145990,0.040454,0.327948,-0.153233,0.568860
5,0.160577,0.424869,-0.110945,0.191033,0.265074,0.501161,-0.215904,-0.589369,-0.173082,0.196957,...,0.104389,-0.075008,-0.218971,-0.654990,-0.012569,-0.005203,0.009601,-0.480887,-0.089556,0.349901
6,-0.211155,0.255086,-0.367761,0.909073,0.304671,0.381588,0.827917,-0.476106,0.017906,-0.346596,...,-0.324946,-0.300407,0.077900,-0.160863,0.136333,-0.330504,-0.243145,-0.367247,-1.031422,0.187445
7,0.071684,0.350215,-0.244403,-0.640559,-0.466073,-0.465960,0.524605,-0.074416,0.777281,0.052274,...,0.417310,0.005921,-0.214619,-0.670994,0.000104,0.762255,0.364880,0.489330,-0.336539,0.542976
8,0.284128,-0.025980,-0.460626,0.184742,0.134648,0.430071,0.022102,-0.036519,0.141513,0.195419,...,-0.075778,-0.548331,-0.207538,-0.293387,-0.647761,0.146201,0.183285,0.255838,-0.044324,-0.118579
9,0.693649,0.273983,-0.567578,0.206454,-0.040428,0.373651,0.341942,0.259474,0.606898,-0.690194,...,0.137069,0.131659,-0.338896,-0.123180,0.030615,0.567973,-0.275404,0.018920,0.469586,0.271939


In [122]:
#Count Vectorizer
cv = CountVectorizer(binary=True) 
cv.fit(interviews_train_clean) #fit the data
X = cv.transform(interviews_train_clean) 

In [123]:
df_sub["gender"] = df_sub["gender"].astype('category')
df_sub.dtypes
df_sub["gender"] = df_sub["gender"].cat.codes
df_sub.head()

,filepath,age,gender,mmse,pause1,pause2,pause3,count_unintelligible,count_trailing,count_repetitions,category,data
0,138-1.cha,68.0,0,28.0,0,0,1,0,1,3,Control,there's a cookie jar on the shelf . and the ...
1,631-0.cha,74.0,1,29.0,0,0,0,0,0,0,Control,the kids are in the cookies . the stool is fa...
2,182-3.cha,NaN,1,NaN,0,0,0,0,1,0,Control,well Johnny's falling off the stool . the boy...
3,121-0.cha,70.0,0,30.0,0,1,0,0,0,0,Control,the boy is taking a cookie outof the cookie ...
4,142-3.cha,61.0,0,30.0,0,0,0,0,0,0,Control,well the water's running over on the floor ....


In [124]:
age_mean = df_sub["age"].mean()
df_sub

,filepath,age,gender,mmse,pause1,pause2,pause3,count_unintelligible,count_trailing,count_repetitions,category,data
0,138-1.cha,68.0,0,28.0,0,0,1,0,1,3,Control,there's a cookie jar on the shelf . and the ...
1,631-0.cha,74.0,1,29.0,0,0,0,0,0,0,Control,the kids are in the cookies . the stool is fa...
2,182-3.cha,NaN,1,NaN,0,0,0,0,1,0,Control,well Johnny's falling off the stool . the boy...
3,121-0.cha,70.0,0,30.0,0,1,0,0,0,0,Control,the boy is taking a cookie outof the cookie ...
4,142-3.cha,61.0,0,30.0,0,0,0,0,0,0,Control,well the water's running over on the floor ....
5,086-2.cha,NaN,0,NaN,1,1,0,0,0,1,Control,well the boy has climbed up on the stool to g...
6,267-2.cha,54.0,0,30.0,2,0,0,0,0,1,Control,mother is drying the dishes and looking out t...
7,118-4.cha,NaN,0,NaN,1,0,0,0,0,0,Control,well the water's running over outta out the s...
8,166-2.cha,NaN,1,NaN,1,0,0,0,0,0,Control,all of the things that I see going on ? well ...
9,105-1.cha,57.0,0,27.0,0,0,0,0,0,0,Control,well the mother is drying the dishes . the s...


In [125]:
df_sub[df_sub.isnull().any(axis=1)].shape

(82, 12)

In [126]:
df_sub.fillna({"age": age_mean}, inplace=True)

In [127]:
df_sub

,filepath,age,gender,mmse,pause1,pause2,pause3,count_unintelligible,count_trailing,count_repetitions,category,data
0,138-1.cha,68.000000,0,28.0,0,0,1,0,1,3,Control,there's a cookie jar on the shelf . and the ...
1,631-0.cha,74.000000,1,29.0,0,0,0,0,0,0,Control,the kids are in the cookies . the stool is fa...
2,182-3.cha,68.155779,1,NaN,0,0,0,0,1,0,Control,well Johnny's falling off the stool . the boy...
3,121-0.cha,70.000000,0,30.0,0,1,0,0,0,0,Control,the boy is taking a cookie outof the cookie ...
4,142-3.cha,61.000000,0,30.0,0,0,0,0,0,0,Control,well the water's running over on the floor ....
5,086-2.cha,68.155779,0,NaN,1,1,0,0,0,1,Control,well the boy has climbed up on the stool to g...
6,267-2.cha,54.000000,0,30.0,2,0,0,0,0,1,Control,mother is drying the dishes and looking out t...
7,118-4.cha,68.155779,0,NaN,1,0,0,0,0,0,Control,well the water's running over outta out the s...
8,166-2.cha,68.155779,1,NaN,1,0,0,0,0,0,Control,all of the things that I see going on ? well ...
9,105-1.cha,57.000000,0,27.0,0,0,0,0,0,0,Control,well the mother is drying the dishes . the s...


In [128]:
X_df = pd.DataFrame(X.todense())
df_sub = df_sub[["age","gender","pause1", "pause2", "pause3", "count_unintelligible", "count_trailing", "count_repetitions", "category"]]
df_sub.reset_index(drop=True, inplace=True)
X = df_sub.join(X_df)

In [129]:
X

,age,gender,pause1,pause2,pause3,count_unintelligible,count_trailing,count_repetitions,category,0,...,1681,1682,1683,1684,1685,1686,1687,1688,1689,1690
0,68.000000,0,0,0,1,0,1,3,Control,1,...,0,0,0,0,0,0,0,0,0,0
1,74.000000,1,0,0,0,0,0,0,Control,0,...,0,0,0,0,0,0,0,0,0,0
2,68.155779,1,0,0,0,0,1,0,Control,0,...,0,0,0,0,0,0,0,0,0,0
3,70.000000,0,0,1,0,0,0,0,Control,0,...,0,0,0,0,0,0,0,0,0,0
4,61.000000,0,0,0,0,0,0,0,Control,0,...,0,0,0,0,0,0,0,0,0,0
5,68.155779,0,1,1,0,0,0,1,Control,0,...,0,0,0,0,0,0,0,0,0,0
6,54.000000,0,2,0,0,0,0,1,Control,0,...,0,0,0,0,0,0,0,0,0,0
7,68.155779,0,1,0,0,0,0,0,Control,1,...,0,0,0,0,0,0,0,0,0,0
8,68.155779,1,1,0,0,0,0,0,Control,1,...,0,0,0,0,0,0,0,0,0,0
9,57.000000,0,0,0,0,0,0,0,Control,0,...,0,0,0,0,0,0,0,0,0,0


In [130]:
X = elmovecs_df.join(X)

In [131]:
X = docvecs_df.join(X)

In [132]:
X

,docvecs0,docvecs1,docvecs2,docvecs3,docvecs4,docvecs5,docvecs6,docvecs7,docvecs8,docvecs9,...,1681,1682,1683,1684,1685,1686,1687,1688,1689,1690
0,-0.151947,-0.371280,-0.316788,0.218896,0.075059,0.085084,-0.014939,-0.332905,0.460935,-0.375095,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.027595,-0.244941,-0.015700,0.170830,0.000994,0.092973,-0.060648,-0.317840,-0.181011,0.074971,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.366985,0.126893,-0.169838,0.375112,-0.321718,0.236652,0.260350,-0.123940,0.463862,-0.413141,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.254116,0.496882,-0.460124,0.546128,-0.271839,0.488808,0.580052,-0.058533,0.128703,0.862113,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.267923,0.645562,0.122693,-0.173702,0.362269,0.026052,0.366241,-0.383295,-0.076927,-0.022577,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.160577,0.424869,-0.110945,0.191033,0.265074,0.501161,-0.215904,-0.589369,-0.173082,0.196957,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,-0.211155,0.255086,-0.367761,0.909073,0.304671,0.381588,0.827917,-0.476106,0.017906,-0.346596,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.071684,0.350215,-0.244403,-0.640559,-0.466073,-0.465960,0.524605,-0.074416,0.777281,0.052274,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.284128,-0.025980,-0.460626,0.184742,0.134648,0.430071,0.022102,-0.036519,0.141513,0.195419,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.693649,0.273983,-0.567578,0.206454,-0.040428,0.373651,0.341942,0.259474,0.606898,-0.690194,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [133]:
y_list = df_sub['category'] #make a list of categories
y_list.shape

(477,)

In [134]:
#train test split and its function with the parameter # of times you want to see the validation accuracy
#also prints out maximum accuracy and its corresponding c each term, mean, and standard deviation
target = [1 if i == 'ProbableAD' else 0 for i in y_list] #这里要写啥
def run_classification_model(num_of_runs, X, target): #the function returns a list of test accuracy
    test_accuracy = [] #the list of accuracy should be empty before each time the function runs
    auc_list = []
    plt.figure(dpi = 800)
    for i in range(num_of_runs): #for loop that runs exact number of times requested
        print("run " + str(i)) #print out which run it is
        validation_accuracy = [] #the list of validation accuracy, the max will be drawn from it at the end
        X_train, X_test, y_train, y_test = train_test_split( #get a test set of 10% of the data, and a train set of 90%
            X, target, test_size = 0.1
        )

        X_train, X_val, y_train, y_val = train_test_split( #get a train set of 80%, and a validation set of 10%
            X_train, y_train, train_size = 0.89
        )
        
        c_list = [0.099, 0.01, 0.05, 0.25, 0.5, 1, 10, 100, 1000, 1e10] #list of hyperparameters
        for c in c_list: #test each c

            lr = LogisticRegression(C=c) #imput the c
            lr.fit(X_train, y_train) #fitting
            
            
            current_validation_accuracy = accuracy_score(y_val, lr.predict(X_val)) #calculate validation accuracy
            validation_accuracy.append(current_validation_accuracy) #append the list
            
        max_accuracy = max(validation_accuracy) #draw the maximum of validation accuracy
        index = np.argmax(validation_accuracy) #take the index of the maximum
        c_optim = c_list[index] #take the corresponding c
        print ("max validation accuracy is {} and optimal c is {}".format(max_accuracy, c_optim)) #print out the results
 
        # apply the best c to train the model
        lr_optim = LogisticRegression(C=c_optim) #try the best c on test set
        lr_optim.fit(X_train, y_train) #fitting
        current_test_accuracy = accuracy_score(y_test, lr_optim.predict(X_test)) #calculate test accuracy
        test_accuracy.append(current_test_accuracy) #append test accuracy list
        
        #plot AUC
        y_pred_proba = lr_optim.predict_proba(X_test)[::,1]
        fpr, tpr, _ = metrics.roc_curve(y_test, y_pred_proba)
        auc = metrics.roc_auc_score(y_test, y_pred_proba)
        auc_list.append(auc)
        plt.plot(fpr,tpr,label="ROC curve, AUC="+str(auc))   

    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Area Under the Curve for AD and healthy control classification')
    plt.legend(loc=4)
    plt.savefig("elmo_roc.png", dpi = 800)
    plt.show()
    
    return test_accuracy, auc_list #return the list

test_accuracy, auc_list = run_classification_model(1000, X.drop("category", axis=1), target) #call the function
mean_accuracy = np.mean(test_accuracy) #take the mean of test accuracy
std_accuracy = np.std(test_accuracy) #take standard deviation
print("Test Accuracy:",test_accuracy)
print("Test Accuracy Mean:",mean_accuracy)
print("Test Accuracy Std:",std_accuracy)

run 0


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

<Figure size 4800x3200 with 0 Axes>

In [29]:
print(np.mean(auc_list))
print(np.std(auc_list))

0.9653172840401747
0.02170583556942998
